In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Mod16_Challenge").getOrCreate()

In [3]:
# Read in data from S3 buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [4]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show(3)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(3)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18RVCKGH1SSI9|          5|            0|          0|   N|                Y|
|R3L4L6LW1PUOFY|          5|            0|          1|   N|                Y|
|R2J8AWXWTDX2TF|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 3 rows



In [22]:
# 2.1 Filter data to retrieve rows where total votes is greater than or equal to 20
total_votes_df = vine_df.filter("total_votes >= 20")
total_votes_df.show(3)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 3 rows



In [24]:
# 2.2 FIlter total_votes_df to retrieve helpful_votes/total votes >= 50%
vine_useful_votes_df = total_votes_df.filter("helpful_votes/total_votes >= .5")
vine_useful_votes_df.show(3)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 3 rows



In [29]:
# 2.3 Filter previous df  to retrieve where review was part of Vine program(paid)
vine_paid_votes_df = vine_useful_votes_df.filter(vine_useful_votes_df["vine"] == "Y")
vine_paid_votes_df.show(3)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3JIGR122X50ZV|          5|         2280|       2314|   Y|                N|
|R3KV8P8WLD2KKC|          5|          263|        286|   Y|                N|
|R2LFY2ZPJ3J7RH|          4|           36|         38|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 3 rows



In [30]:
# 2.4 Filter previous df  to retrieve where review was NOT part of Vine program(paid)
vine_unpaid_votes_df = vine_useful_votes_df.filter(vine_useful_votes_df["vine"] == "N")
vine_unpaid_votes_df.show(3)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 3 rows



In [33]:
# 2.5 Determine number of reviews, number of 5 star reviews, and percentage of 5 star reviews for the paid vs unpaid reviews
      # number of reviews paid
total_count_paid = vine_paid_votes_df.count()
total_count_paid

969

In [38]:
      # number of 5 star reviews paid
total_5star_paid = vine_paid_votes_df.filter("star_rating = 5").count()
total_5star_paid

430

In [48]:
      # percentage of 5 star reviews paid
fivestar_vs_total_perc = (total_5star_paid/total_count_paid)*100
fivestar_vs_total_perc

44.375644994840044

In [45]:
      # number of reviews unpaid
total_count_unpaid = vine_unpaid_votes_df.count()
total_count_unpaid

43745

In [46]:
      # number of 5 star reviews unpaid
total_5star_unpaid = vine_unpaid_votes_df.filter("star_rating = 5").count()
total_5star_unpaid

19233

In [49]:
      # percentage of 5 star reviews unpaid
unpaid_fivestar_vs_total_perc = (total_5star_unpaid/total_count_unpaid)*100
unpaid_fivestar_vs_total_perc

43.96616756200709

In [52]:
# total percentage of 5-star
total_5star_paid/(total_5star_paid+total_5star_unpaid)*100

2.186848395463561